In [253]:
! pip3 install fasttext
! pip3 install nltk
! pip3 install snowballstemmer
! pip3 install sklearn
! pip3 install networkx

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [254]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import re
from snowballstemmer import TurkishStemmer
import string
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import fasttext
import fasttext.util
fasttext.util.download_model('tr', if_exists='ignore')
ft = fasttext.load_model('cc.tr.300.bin')

In [255]:
def stem_it(x):
    """Turkish stemmer.
    Returns stemmed words in given string, uses Snowball Turkish stemmer."""
    return [turkStem.stemWord(w) for w in x.split()]

def remove_digit(text):
    """Digit cleaner.
    Removes digits in given string."""
    digit_pattern = str.maketrans('', '', string.digits)
    return text.translate(digit_pattern)

def lower_it(text):
    """Lowers characters in given string.
    Lowers all in string."""
    return text.lower()

def remove_punctiations(text):
    """Punction cleaner.
    Removes punctions in given string."""
    pattern = str.maketrans('', '', string.punctuation)
    text= text.translate(pattern)
    return text.replace("\n", " ")

def remove_stopwords(text, language='turkish'):
    """Stopword cleaner.
    Removes stopwords in given string, language is set to Turkish by default."""
    st = stopwords.words(language)
    pattern = re.compile(r'\b(' + r'|'.join(st) + r')\b\s*')
    text = pattern.sub('',text)
    return text

def replace_turkish(text):
    """Turkish Character Fixer.
    Replaces Turkish character with English ones."""
    character_map = {'ç': 'c',
                     'Ç': 'C',
                     'ğ': 'g',
                     'Ğ': 'G',
                     'ı': 'i',
                     'I': 'I',
                     'ö': 'o',
                     'Ö': 'O',
                     'ş': 's',
                     'Ş': 'S', 
                     'ü': 'u',
                     'Ü': 'U'}
    for initial, final in character_map.items():
        text = text.replace(initial, final)
    return text

def sentence_it(x):
    """Sentence maker.
    Returns one string form of given words array."""
    sentence = ''
    for w in x.split(): sentence += w + " "
    return sentence.strip()


In [256]:

def preproces(text, stemmer=False):
    """String cleaner.
    Removes digits, lowers charactes, removes punctiations and stop words in given string."""
    text = remove_digit(text)
    text = lower_it(text)
    text = remove_punctiations(text)
    text = remove_stopwords(text)
    #text = replace_turkish(text) # Since FastText includes Turkish characters, there is no need to replace them.
    # But for different libraries, one may have to change them.
    if stemmer:
        text = stem_it(text)
        text = sentence_it(text)
    text = sentence_it(text)
    return text

In [257]:
def text_cleaning(text):
    """Text cleaner.
    Returns sentences and cleaned sentences."""
    sentences = sent_tokenize(text)
    clean_sentenes = [preproces(sentence) for sentence in sentences]
    return sentences, clean_sentenes 

In [258]:
def sentence_vector(sentence):
    return sum([ft.get_word_vector(word) for word in sentence])/(len(sentence)+0.0001)

In [259]:
def word_vectors(text):
    """Word vector creator.
    Uses cleaned text array and returns word vector array"""
    sentence_words = [[w for w in s.split()] for s in text]
    word_vectors = [sentence_vector(s_words) for s_words in sentence_words]
    return word_vectors

In [260]:
def similarity_matrix(text_vector):
    """Similarty Matrix generator.
    Creates similarity matrix using sentence vectors via cosine similarity"""
    sim_mat = np.zeros([len(text_vector), len(text_vector)])
    for i in range(len(text_vector)):
        for j in range(len(text_vector)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(text_vector[i].reshape(1, -1), text_vector[j].reshape(1, -1))[0, 0]
    return sim_mat

In [261]:
def extraction(sentences, similarity_matrix):
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    text_dict = {}
    for i in range(len(sentences)):
        text_dict.update({str(i):{"score": scores[i],
                                  "text": sentences[i]}})
    text_order = sorted(text_dict, key=lambda x: text_dict[x]["score"], reverse=True)
    extract = ""
    for i in range(int(len(sentences)/3)):
        id = text_order[i]
        extract = extract + " " + text_dict[id]["text"]
    return extract

In [262]:
def generate_extraction(text):
    sentences, cleaned_sentences = text_cleaning(text)
    sentence_vectors = word_vectors(cleaned_sentences)
    sim_mat = similarity_matrix(sentence_vectors)
    return extraction(sentences, sim_mat)

In [263]:
test_text = """CHP Saray İlçe Başkan Vekili Yasin Arslan, Saray İlçe Seçim Kurulu Başkanlığına 31 Mart'ta yüzde 51,16 oy alan DEM Parti adayı Davut Acar ile ilgili itiraz dilekçesi verdi. İlçe Seçim Kurulu talebi reddederken gelişme üzerine CHP Van İl Başkanlığı açıklamada bulunarak Arslan'ın istifa ettiğini ancak yine de kesin ihraç talebiyle disipline sevk edildiğini bildirdi.

"TERÖR ARAŞTIRMASI YAPILSIN"

Arslan, DEM Partili adayın başkan seçilmesine itiraz ettiği dilekçesinde şu ifadeleri kullandı: "31 Mart tarihinde yapılan yerel seçimlerde Van ili Saray ilçesinde Cumhuriyet Halk Partisi olarak belediye başkanlık seçimlerine katılmış bulunmaktayız. Söz konusu seçim sonuçlarında rakip parti (DEM Parti) adayı Davut Acar'ın seçilmesi önünde engel teşkil edebilecek terörle ilişkili davalarda yargılanıp yargılanmadığı, yargılanmış ise memnu haklarından mahrum bırakılıp bırakılmadığı, devlet memuru vasıflarının taşıyıp taşımadığı hususlarında araştırmaların yapılarak, bu durum söz konusu ise mazbatasının iptalini bilgilerinize arz ederim."

İlçe Seçim Kurulu Başkanlığı Acar hakkında "kesinleşmiş bir mahkumiyet hükmü bulunmadığından" itiraz yolu açık olmak üzere talebin reddine karar verdi.
CHP İL BAŞKANLIĞI: KESİN İHRAÇ TALEBİYLE DİSİPLİNE SEVK EDİLDİ

CHP Van İl Başkanlığından bugün yapılan açıklamada, "Saray ilçemizde il başkanlığımız bilgisi dışında İlçe Seçim Kuruluna bir müracaat gerçekleştiğini üzüntü ile öğrendik. İlçe Başkan Vekili Yasin Arslan İlçe Seçim Kuruluna verdiği dilekçenin ardından bir de istifa dilekçesi vermiştir. Bunun üzerine harekete geçen il başkanlığımız ilgili müracaatı yapan İlçe Başkan Vekilinin sadece istifasını yeterli bulmamış kendisini kesin ihraç talebiyle il disiplin kuruluna sevk etmiştir. Halkın iradesinin dışında bir irade olamayacağını, bu vesileyle tekrar tüm siyasi partileri halkın iradesine saygılı olmaya davet ettiğimizi tüm kamuoyuna saygılarımızla arz ve beyan ederiz" ifadelerine yer verildi. """

In [264]:
generate_extraction(test_text)

' İlçe Seçim Kurulu talebi reddederken gelişme üzerine CHP Van İl Başkanlığı açıklamada bulunarak Arslan\'ın istifa ettiğini ancak yine de kesin ihraç talebiyle disipline sevk edildiğini bildirdi. "TERÖR ARAŞTIRMASI YAPILSIN"\n\nArslan, DEM Partili adayın başkan seçilmesine itiraz ettiği dilekçesinde şu ifadeleri kullandı: "31 Mart tarihinde yapılan yerel seçimlerde Van ili Saray ilçesinde Cumhuriyet Halk Partisi olarak belediye başkanlık seçimlerine katılmış bulunmaktayız. İlçe Başkan Vekili Yasin Arslan İlçe Seçim Kuruluna verdiği dilekçenin ardından bir de istifa dilekçesi vermiştir.'

In [265]:
import requests

In [266]:
tt = "https://newsapi.org/v2/everything?q=tesla&language=tr&from=2024-03-11&sortBy=publishedAt&pageSize=100&page=1&apiKey=9f8f599f440c406fb0b8a0d28ad439ce"


In [267]:
response = requests.get(tt)

In [272]:
r = response.json()

In [273]:
print(r)

{'status': 'ok', 'totalResults': 193, 'articles': [{'source': {'id': None, 'name': 'Shiftdelete.net'}, 'author': 'Alperen Esin', 'title': 'Tesla, otopilot kazasında kusurlu bulundu!', 'description': 'Bu içerik ilk olarak Tesla, otopilot kazasında kusurlu bulundu! adresinde yayınlandı Teknoloji Haberleri - ShiftDelete.Net.', 'url': 'https://shiftdelete.net/tesla-otopilot-davasi-kusurlu-bulundu', 'urlToImage': 'https://ares.shiftdelete.net/2023/08/tesla-veri-sizintisi.jpg', 'publishedAt': '2024-04-10T11:15:00Z', 'content': 'Tesla, 2018 ylnda meydana gelen ve Apple mühendisi Walter Huang’n yaamn yitirdii otopilot kazasnda kusurlu bulundu. Kazada Huang Model X aracyla otopilot modunda ilerlerken bariyerlere çarpm, bunun a… [+1449 chars]'}, {'source': {'id': None, 'name': 'Donanimhaber.com'}, 'author': 'Deniz Çakmak', 'title': 'Otopilot davasında, Tesla ve ölen mühendisin ailesi uzlaştı: Tesla ödeme yapacak', 'description': "Tesla, Apple mühendisi Walter Huang'ın Model X aracıyla otopilot i

In [275]:
r["articles"][0]

{'source': {'id': None, 'name': 'Shiftdelete.net'},
 'author': 'Alperen Esin',
 'title': 'Tesla, otopilot kazasında kusurlu bulundu!',
 'description': 'Bu içerik ilk olarak Tesla, otopilot kazasında kusurlu bulundu! adresinde yayınlandı Teknoloji Haberleri - ShiftDelete.Net.',
 'url': 'https://shiftdelete.net/tesla-otopilot-davasi-kusurlu-bulundu',
 'urlToImage': 'https://ares.shiftdelete.net/2023/08/tesla-veri-sizintisi.jpg',
 'publishedAt': '2024-04-10T11:15:00Z',
 'content': 'Tesla, 2018 ylnda meydana gelen ve Apple mühendisi Walter Huang’n yaamn yitirdii otopilot kazasnda kusurlu bulundu. Kazada Huang Model X aracyla otopilot modunda ilerlerken bariyerlere çarpm, bunun a… [+1449 chars]'}